In [1]:
# 作者：MiloMusiala
# 链接：https://leetcode-cn.com/problems/evaluate-division/solution/pythonbing-cha-ji-fu-mo-ban-by-milomusia-kfsu/
# 来源：力扣（LeetCode）
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

# [并查集](https://leetcode-cn.com/problems/number-of-provinces/solution/python-duo-tu-xiang-jie-bing-cha-ji-by-m-vjdr/)

## 基本概念

* 并查集是一种数据结构
* 并查集这三个字，一个字代表一个意思。
* 并（Union），代表合并
* 查（Find），代表查找
* 集（Set），代表这是一个以字典为基础的数据结构，它的基本功能是合并集合中的元素，查找集合中的元素
* 并查集的典型应用是有关连通分量的问题
* 并查集解决单个问题（添加，合并，查找）的时间复杂度都是O(1)。因此，并查集可以应用到在线算法中

## 并查集的实现

数据结构：并查集跟树有些类似，只不过她跟树是相反的。在树这个数据结构里面，每个节点会记录它的子节点。在并查集里，每个节点会记录它的父节点。

<img src="https://pic.leetcode-cn.com/1609980000-ofFjdW-%E5%B9%BB%E7%81%AF%E7%89%871.JPG" width=50%>

可以看到，如果节点是相互连通的（从一个节点可以到达另一个节点），那么他们在同一棵树里，或者说在同一个集合里，或者说他们的祖先是相同的。

## 初始化
当把一个新节点添加到并查集中，它的父节点应该为空

<img src="https://pic.leetcode-cn.com/1609980044-MZZZkZ-%E5%B9%BB%E7%81%AF%E7%89%872.JPG" width=50%>

## 合并两个节点
如果发现两个节点是连通的，那么就要把他们合并，也就是他们的祖先是相同的。这里究竟把谁当做父节点一般是没有区别的。

<img src="https://pic.leetcode-cn.com/1609980079-JmfIbX-%E5%B9%BB%E7%81%AF%E7%89%874.JPG" width=50%>

## 两节点是否连通
我们判断两个节点是否处于同一个连通分量的时候，就需要判断它们的祖先是否相同

## 查找祖先
查找祖先的方法是：如果节点的父节点不为空，那就不断迭代。

<img src="https://pic.leetcode-cn.com/1609980128-GpUyoj-%E5%B9%BB%E7%81%AF%E7%89%877.JPG" width=50%>

这里有一个优化的点：如果我们树很深，比如说退化成链表，那么每次查询的效率都会非常低。所以我们要做一下路径压缩。也就是把树的深度固定为二。

<img src="https://pic.leetcode-cn.com/1609980188-XcJBuX-%E5%B9%BB%E7%81%AF%E7%89%8711.JPG" width=50%>

这么做可行的原因是，并查集只是记录了节点之间的连通关系，而节点相互连通只需要有一个相同的祖先就可以了。

路径压缩可以用递归，也可以迭代。这里用迭代的方法。


# 399. 除法求值

给你一个变量对数组equations和一个实数值数组values作为已知条件，其 equations\[i] =\[Ai,Bi]和values\[i]共同表示等式 Ai/Bi = values\[i]。每个Ai或Bi是一个表示单个变量的字符串。

另有一些以数组queries表示的问题，其中queries\[j]=\[Cj,Dj]表示第j个问题，请你根据已知条件找出Cj/Dj =?的结果作为答案。

返回所有问题的答案 。如果存在某个无法确定的答案，则用 -1.0 替代这个答案。如果问题中出现了给定的已知条件中没有出现的字符串，也需要用 -1.0 替代这个答案。

注意：输入总是有效的。你可以假设除法运算中不会出现除数为 0 的情况，且不存在任何矛盾的结果。


In [6]:
from solution import UnionFind

class UFind(UnionFind):
    def __init__(self):
        UnionFind.__init__(self)
        self.father = {}
        self.value = {}

    def add(self, x):
        if x not in self.father:
            self.father[x] = None
            self.value[x] = 1.0

    def merge(self, x, y, val):
        root_x, root_y = self.find(x), self.find(y)
        if root_x != root_y:
            self.father[root_x] = root_y
            self.value[root_x] = self.value[y] * val / self.value[x]

    def find(self,x):
        """
        查找根节点 路径压缩 更新权重
        """
        root = x        
        base = 1  # 节点更新权重的时候要放大的倍数
        while self.father[root] != None:
            root = self.father[root]
            base *= self.value[root]
        while x != root:
            original_father = self.father[x]           
            self.value[x] *= base  # 离根节点越远，放大的倍数越高
            base /= self.value[original_father]
            self.father[x] = root
            x = original_father         
        return root
    
    def is_connected(self,x,y):
        """
        两节点是否相连
        """
        return x in self.value and y in self.value and self.find(x) == self.find(y)

class Solution:
    def calcEquation(self, equations, values, queries):
        '''
            equations: List[List[str]]
            values: List[float], 
            queries: List[List[str]]) 
            out: List[float]
        '''
        uf = UnionFind()
        for (a,b),val in zip(equations,values):
            uf.add(a)
            uf.add(b)
            uf.merge(a,b,val)
        
        res = [-1.0] * len(queries)
        print("单纯的并集：", uf.father, uf.value)
        '''
            在以上的步骤当中，主要是将两两之间的节点联系起来，
            如果添加的节点当中有以前的子节点，那么更新该节点
            到根节点之间的节点的根节点，而不更新该节点的子节
            点的根节点。
            在下面的步骤中，判断两个节点是否有相同的根节点，
            判断过程中更新该节点与根节点之间节点的根节点。
        '''
        for i,(a,b) in enumerate(queries):
            if uf.is_connected(a,b):
                res[i] = uf.value[a] / uf.value[b]
        print("查找更新之后：", uf.father, uf.value)
        return res

equations = [["a", "b"], ["b", "c"]]
values = [2.0, 3.0]
queries = [["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], ["x", "x"]]

sol = Solution()
results = sol.calcEquation(equations, values, queries)
print("results:",results)


单纯的并集： {'a': 'b', 'b': 'c', 'c': None} {'a': 2.0, 'b': 3.0, 'c': 1.0}
查找更新之后： {'a': 'c', 'b': 'c', 'c': None} {'a': 6.0, 'b': 3.0, 'c': 1.0}
results: [6.0, 0.5, -1.0, 1.0, -1.0]


# 547. 省份数量

有 n 个城市，其中一些彼此相连，另一些没有相连。如果城市 a 与城市 b 直接相连，且城市 b 与城市 c 直接相连，那么城市 a 与城市 c 间接相连。

省份 是一组直接或间接相连的城市，组内不含其他没有相连的城市。

给你一个 n x n 的矩阵 isConnected ，其中 isConnected\[i]\[j] = 1 表示第 i 个城市和第 j 个城市直接相连，而 isConnected\[i]\[j] = 0 表示二者不直接相连。

返回矩阵中 省份 的数量。

### 个人思路:
省份即互不联通的单位,能联通的节点并到一个集合中,也就是拥有相同的根节点,最终判断有多少个根节点

### 官方思路
可以把n个城市和它们之间的相连关系看成图，城市是图中的节点，相连关系是图中的边，给定的矩阵isConnected即为图的邻接矩阵，省份即为图中的连通分量。

计算省份总数，等价于计算图中的连通分量数，可以通过深度优先搜索或广度优先搜索实现，也可以通过并查集实现。

## 方法一：深度优先搜索
深度优先搜索的思路是很直观的。遍历所有城市，对于每个城市，如果该城市尚未被访问过，则从该城市开始深度优先搜索，通过矩阵isConnected得到与该城市直接相连的城市有哪些，这些城市和该城市属于同一个连通分量，然后对这些城市继续深度优先搜索，直到同一个连通分量的所有城市都被访问到，即可得到一个省份。遍历完全部城市以后，即可得到连通分量的总数，即省份的总数。

## 方法二:并查集
计算连通分量数的另一个方法是使用并查集。初始时，每个城市都属于不同的连通分量。遍历矩阵isConnected，如果两个城市之间有相连关系，则它们属于同一个连通分量，对它们进行合并。遍历矩阵isConnected 的全部元素之后，计算连通分量的总数，即为省份的总数。



In [11]:
from solution import UnionFind

class Solution(object):
    def findCircleNum(self, isConnected):
        """
        :type isConnected: List[List[int]]
        :rtype: int
        """
        '''方法一(self):使用并查集'''
        # uf = UnionFind()
        # numCity = len(isConnected)
        # for i in range(numCity):
        #     uf.add(i)
        # for j in range(numCity):
        #     for i in range(j+1, numCity):
        #         if isConnected[j][i] == 1:
        #             uf.merge(j, i)
        # print(uf.father)
        # ans = 0
        # for key, val in uf.father.items():
        #     if val == None: ans += 1
        # return ans

        '''方法一(官方):并查集
        灵活运用并查集,并不是每次都要将整个类写上去
        时间复杂度:O(n^2logn) 空间复杂度:O(n)
        '''
        def find(index: int) -> int:
            if parent[index] != index:
                parent[index] = find(parent[index])
            return parent[index]
        def union(index1: int, index2: int):
            parent[find(index1)] = find(index2)
        provinces = len(isConnected)
        parent = list(range(provinces))
        for i in range(provinces):
            for j in range(i + 1, provinces):
                if isConnected[i][j] == 1:
                    union(i, j)
        print(parent)
        circles = sum(parent[i] == i for i in range(provinces))
        return circles


        '''方法二(官方):深度优先搜索
        时间复杂度:O(n^2) 空间复杂度:O(n)
        '''
        def dfs(i: int):
            for j in range(provinces):
                if isConnected[i][j] == 1 and j not in visited:
                    visited.add(j)
                    dfs(j)
        provinces = len(isConnected)
        visited = set()
        circles = 0
        for i in range(provinces):
            if i not in visited:
                dfs(i)
                circles += 1
        return circles

        '''方法三(官方):广度优先搜索
        对于每个城市，如果该城市尚未被访问过，则从该城市开始广度优先搜索，
        直到同一个连通分量中的所有城市都被访问到，即可得到一个省份。
        时间复杂度:O(n^2) 空间复杂度:O(n)
        '''
        provinces = len(isConnected)
        visited = set()
        circles = 0
        for i in range(provinces):
            if i not in visited:
                Q = collections.deque([i])
                while Q:
                    j = Q.popleft()
                    visited.add(j)
                    for k in range(provinces):
                        if isConnected[j][k] == 1 and k not in visited:
                            Q.append(k)
                circles += 1   
        return circles
        

isConnected = [[1,1,0],[1,1,0],[0,0,1]]
isConnected = [[1,0,1,1,0],
               [0,1,1,1,0],
               [1,1,1,1,0],
               [1,1,1,1,0],
               [0,0,0,0,1],]
sol = Solution()
print(sol.findCircleNum(isConnected))


[2, 3, 3, 3, 4]
2


# 684. [冗余连接](https://leetcode-cn.com/problems/redundant-connection/)

在本问题中, 树指的是一个连通且无环的无向图。

输入一个图，该图由一个有着N个节点 (节点值不重复1, 2, ..., N) 的树及一条附加的边构成。附加的边的两个顶点包含在1到N中间，这条附加的边不属于树中已存在的边。

结果图是一个以边组成的二维数组。每一个边的元素是一对\[u, v] ，满足 u < v，表示连接顶点u 和v的无向图的边。

返回一条可以删去的边，使得结果图是一个有着N个节点的树。如果有多个答案，则返回二维数组中最后出现的边。答案边 \[u, v] 应满足相同的格式 u < v。

## 这道题我都读不懂啊

## 方法一:并查集

在一棵树中，边的数量比节点的数量少1。如果一棵树有N个节点，则这棵树有N−1条边。这道题中的图在树的基础上多了一条附加的边，因此边的数量也是N。树是一个连通且无环的无向图，在树中多了一条附加的边之后就会出现环，因此附加的边即为导致环出现的边。

可以通过并查集寻找附加的边。初始时，每个节点都属于不同的连通分量。遍历每一条边，判断这条边连接的两个顶点是否属于相同的连通分量。如果两个顶点属于不同的连通分量，则说明在遍历到当前的边之前，这两个顶点之间不连通，因此当前的边不会导致环出现，合并这两个顶点的连通分量。如果两个顶点属于相同的连通分量，则说明在遍历到当前的边之前，这两个顶点之间已经连通，因此当前的边导致环出现，为附加的边，将当前的边作为答案返回。



In [45]:
class Solution(object):
    def findRedundantConnection(self, edges):
        """
        :type edges: List[List[int]]
        :rtype: List[int]
        """
        '''方法一(官方):并查集
        时间复杂度:O(NlogN) 空间复杂度:O(N)
        '''
        nodesCount = len(edges)
        parent = list(range(nodesCount + 1))
        # print(parent)
        def find(index: int) -> int:
            if parent[index] != index:
                parent[index] = find(parent[index])
            return parent[index]
        
        def union(index1: int, index2: int):
            parent[find(index1)] = find(index2)

        for node1, node2 in edges:
            if find(node1) != find(node2):
                union(node1, node2)
            else:
                return [node1, node2]
        
        return []

edges = [[1,2], [1,3], [2,3]]
edges = [[1,2], [2,3], [3,4], [1,4], [1,5]]
sol = Solution()
print(sol.findRedundantConnection(edges))

[0, 1, 2, 3, 4, 5]
[1, 4]


# 1319. 连通网络的操作次数(不太懂)
[链接](https://leetcode-cn.com/problems/number-of-operations-to-make-network-connected/)

用以太网线缆将n台计算机连接成一个网络，计算机的编号从0到n-1。线缆用connections表示，其中 connections\[i] = \[a, b]连接了计算机a和b。网络中的任何一台计算机都可以通过网络直接或者间接访问同一个网络中其他任意一台计算机。

给你这个计算机网络的初始布线 connections，你可以拔开任意两台直连计算机之间的线缆，并用它连接一对未直连的计算机。请你计算并返回使所有计算机都连通所需的最少操作次数。如果不可能，则返回 -1 。 



In [54]:
from solution import UnionFind

class UF(UnionFind):
    def __init__(self, n):
        UnionFind.__init__(self)
        self.parent = list(range(n))
        self.size = [1] * n
        self.n = n
        # 当前连通分量数目
        self.setCount = n
    
    def findset(self, x: int) -> int:
        if self.parent[x] == x: return x
        self.parent[x] = self.findset(self.parent[x])
        return self.parent[x]
    
    def unite(self, x: int, y: int) -> bool:
        x, y = self.findset(x), self.findset(y)
        if x == y: return False
        if self.size[x] < self.size[y]: x, y = y, x
        self.parent[y] = x
        self.size[x] += self.size[y]
        self.setCount -= 1
        return True
 
    def connected(self, x: int, y: int) -> bool:
        x, y = self.findset(x), self.findset(y)
        return x == y

class Solution:
    def makeConnected(self, n, connections):
        if len(connections) < n - 1: return -1      
        uf = UF(n)
        for x, y in connections: uf.unite(x, y)
        return uf.setCount - 1

n = 4
connections = [[0,1],[0,2],[1,2]]
n = 6
connections = [[0,1],[0,2],[0,3],[1,2],[1,3]]
sol = Solution()
print(sol.makeConnected(n, connections))

2


# 130. 被围绕的区域

给你一个 m x n 的矩阵 board ，由若干字符 'X' 和 'O' ，找到所有被 'X' 围绕的区域，并将这些区域里所有的 'O' 用 'X' 填充。

![1](https://assets.leetcode.com/uploads/2021/02/19/xogrid.jpg)





In [62]:
class Solution(object):
    def solve(self, board):
        """
        :type board: List[List[str]]
        :rtype: None Do not return anything, modify board in-place instead.
        """
        '''方法一(官方):深度优先搜索
        时间复杂度:O(n×m) 空间复杂度:O(n×m)
        '''
        # if not board: return
        # n, m = len(board), len(board[0])
        # def dfs(x, y): # 使用了一个递归的搜索方法
        #     if not 0 <= x < n or not 0 <= y < m or board[x][y] != 'O': return
        #     board[x][y] = "A"
        #     dfs(x + 1, y)
        #     dfs(x - 1, y)
        #     dfs(x, y + 1)
        #     dfs(x, y - 1)
        
        # for i in range(n):
        #     dfs(i, 0)
        #     dfs(i, m - 1)
        # for i in range(m - 1):
        #     dfs(0, i)
        #     dfs(n - 1, i)
        
        # for i in range(n):
        #     for j in range(m):
        #         if board[i][j] == "A":
        #             board[i][j] = "O"
        #         elif board[i][j] == "O":
        #             board[i][j] = "X"
        '''方法二(官方):广度优先搜索
        时间复杂度:O(n×m) 空间复杂度:O(n×m)
        '''
        if not board: return
        n, m = len(board), len(board[0])
        import collections
        que = collections.deque()
        for i in range(n):
            if board[i][0] == "O": que.append((i, 0))
            if board[i][m - 1] == "O": que.append((i, m - 1))
        for i in range(m - 1):
            if board[0][i] == "O": que.append((0, i))
            if board[n - 1][i] == "O": que.append((n - 1, i))
        while que:
            x, y = que.popleft()
            board[x][y] = "A"
            for mx, my in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
                if 0 <= mx < n and 0 <= my < m and board[mx][my] == "O":
                    que.append((mx, my))
        for i in range(n):
            for j in range(m):
                if board[i][j] == "A": board[i][j] = "O"
                elif board[i][j] == "O": board[i][j] = "X"


board = [["X","X","X","X"],["X","O","O","X"],["X","X","O","X"],["X","O","X","X"]]
sol = Solution()
sol.solve(board)
print(board)

[['X', 'X', 'X', 'X'], ['X', 'X', 'X', 'X'], ['X', 'X', 'X', 'X'], ['X', 'O', 'X', 'X']]


# 200. [岛屿数量](https://leetcode-cn.com/problems/number-of-islands/solution/dao-yu-shu-liang-by-leetcode/)有待进一步

给你一个由 '1'（陆地）和 '0'（水）组成的的二维网格，请你计算网格中岛屿的数量。岛屿总是被水包围，并且每座岛屿只能由水平方向和/或竖直方向上相邻的陆地连接形成。此外，你可以假设该网格的四条边均被水包围。




In [67]:
class UnionFind:
    def __init__(self, grid):
        m, n = len(grid), len(grid[0])
        self.count = 0
        self.parent = [-1] * (m * n)
        self.rank = [0] * (m * n)
        for i in range(m):
            for j in range(n):
                if grid[i][j] == "1":
                    self.parent[i * n + j] = i * n + j
                    self.count += 1
    
    def find(self, i):
        if self.parent[i] != i:
            self.parent[i] = self.find(self.parent[i])
        return self.parent[i]
    
    def union(self, x, y):
        rootx = self.find(x)
        rooty = self.find(y)
        if rootx != rooty:
            if self.rank[rootx] < self.rank[rooty]:
                rootx, rooty = rooty, rootx
            self.parent[rooty] = rootx
            if self.rank[rootx] == self.rank[rooty]:
                self.rank[rootx] += 1
            self.count -= 1
    
    def getCount(self):
        return self.count

class Solution(object):
    def numIslands(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: int
        """
        '''方法一(官方):深度优先搜索
        如果一个位置为 11，则以其为起始节点开始进行深度优先搜索。
        在深度优先搜索的过程中，每个搜索到的1都会被重新标记为 0。
        最终岛屿的数量就是我们进行深度优先搜索的次数。
        时间复杂度:O(MN) 空间复杂度:O(MN)
        '''
        def dfs(grid, r, c):
            grid[r][c] = 0
            nr, nc = len(grid), len(grid[0])
            for x, y in [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]:
                if 0 <= x < nr and 0 <= y < nc and grid[x][y] == "1":
                    dfs(grid, x, y)
        nr = len(grid)
        if nr == 0: return 0
        nc = len(grid[0])
        num_islands = 0
        for r in range(nr):
            for c in range(nc):
                if grid[r][c] == "1":
                    num_islands += 1
                    dfs(grid, r, c)
        return num_islands
        '''方法二(官方):广度优先搜索
        时间复杂度:O(MN) 空间复杂度:O(min(M,N))
        '''
        nr = len(grid)
        if nr == 0:
            return 0
        nc = len(grid[0])

        num_islands = 0
        for r in range(nr):
            for c in range(nc):
                if grid[r][c] == "1":
                    num_islands += 1
                    grid[r][c] = "0"
                    neighbors = collections.deque([(r, c)])
                    while neighbors:
                        row, col = neighbors.popleft()
                        for x, y in [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]:
                            if 0 <= x < nr and 0 <= y < nc and grid[x][y] == "1":
                                neighbors.append((x, y))
                                grid[x][y] = "0"
        return num_islands
        '''方法三(官方):并查集
        时间复杂度:O(MN×α(MN)) 空间复杂度:O(MN)
        '''
        nr = len(grid)
        if nr == 0:
            return 0
        nc = len(grid[0])
        uf = UnionFind(grid)
        num_islands = 0
        for r in range(nr):
            for c in range(nc):
                if grid[r][c] == "1":
                    grid[r][c] = "0"
                    for x, y in [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]:
                        if 0 <= x < nr and 0 <= y < nc and grid[x][y] == "1":
                            uf.union(r * nc + c, x * nc + y)
        
        return uf.getCount()


grid = [
  ["1","1","1","1","0"],
  ["1","1","0","1","0"],
  ["1","1","0","0","0"],
  ["0","0","0","0","0"]
]
sol = Solution()
print(sol.numIslands(grid))

1
